# COVID Incremental Ventilator Requirements

In [6]:
import sys
import pandas as pd
sys.path.append('../src')
from restart import NoteCompose
from data import Data
from util import set_config, to_df, to_sheet, display_population, format_population

## Methodology Overview and Assumptions

Quick overview of the current state of this document: the notebook code here walks you through the methodology and assumptions made.

Code to fetch the IHME and MIT numbers is in `src/extern/data/epidemiological` (written by Ethan@restart.us). We are not using the MIT projections have a peak in August and are monotically decreasing ever since so this does not seem to take into account the fall surge 

In addition, various RAND and CAN scenarios were pulled from the CalCat and placed into the model. Note we are using the high-band estimate for a more "conservative case". The issue is that all these models are very far out in the future and highly speculative because:

1. We are modeling human behavior and there is no history. We do not know how the general population will handling masking.
2. The fall spread. This is the first time we enter the fall with COVID in the US, so there is no historical data.
3. We are layering in more and more speculation. This is quite different than PPE because most PPE estimates are for prophalaxis, but this is trying to model the disease and then estimate the severity. That is much more difficult

The net conclusion is that:
1. We are not modeling the "baseline need" for non-COVID. The flu season typically means that ventilator use and ICU use will increase. We have not found a model that adds these effects in, we do know that in many regions, ICU utilization increases dramatically in the flu season, but the combination of less mobility, more awareness of the flu and the Southern hemisphere experience for flu this year hopefully mean less ICU/ventilator needs.
2. For incremental COVID-19 needs for a combination of hospitals plus the state backstop, it looks like 15-25K ventilators total ventilators needed. The 95% uncertainty level for the IMHE and RAND models which do attempt to model this are at 10K, but because they are modeling specific human behaviors, they may break through those statistical measures. For instance, if masking falls dramatically, beyond the initial assumptions, then the need will rise.
3. The second problem is that ventilators are not as easily moved, so having them in one county may not help an immediate need in another. And the current treatment regimes and severity levels are changing as well, so these studies use historical ventilator usage and this could be much lower if treatments don't need it or much higher if more disease severity increases.
3. This is an early look and based on projections by experts as always this difficult decision has to be made taking into account the worst, worst case on human behavior and the unknown effects of the fall.

In [7]:
config = set_config('../src')
restart = NoteCompose(configdir='../src/', population='dict')
population = restart.model.population
resource = restart.model.resource
demand = restart.model.demand

**Burn Rate Estimates (CDPH and IMHE)**

In [8]:
burn_sheet = format_population(to_sheet(demand.demand_per_unit_map_dn_um.df))
burn_sheet

Sheet(cells=(Cell(choice=[], column_end=0, column_start=0, numeric_format='0,000', read_only=True, row_end=1, …

The CHA memo doe not estimate the ventilator burn rate, which comes from IHME. They simply estimate ICU demand as 27% of all COVID hospitalizations, and make the assumption that 90% of COVID ICU patients will need to go on vent, so (0.27)(0.9) = 0.243. We do not know if these assumptions are valid for California or are appropriate going forward

The rest of the burn rates are derived from the CHA memo (and this model replicates the PPE projections in that memo). The CHA burn rates are higher than other rates we have estimated based on Washington and other data. However, we have not studied these burn rates very much. 

Below are the population estimates we're using, and then also the resulting projections. I've added a range dimension, but not all patient projections contain ranges. In these cases we are using a simple exponential to estimate bands and this needs to be validated.

First are the calculations with no ranges, using an upper bound (or if only one number is given, just that one number).

In [9]:
import ipywidgets as widgets
pop_sheet = format_population(to_sheet(population.population_pP_tr.df), round=True)
daily_sheet = format_population(to_sheet(demand.demand_by_pop_total_pn_tc.df), round=True)

widgets.VBox([pop_sheet, daily_sheet])

Next is the same model but with the added range dimension. Use the slider to adjust the confidence interval, and the projections will automatically update. Note how some of the models give us very high projections for ventilators. This reflects the uncertainty in measurement and means that judgement (which we can't provide) is critical to making this decision

In [10]:
import ipywidgets as widgets
from scipy.stats import norm

epi_df = pd.read_csv('epi_ranges.csv',index_col='Model')
epi_df['Standard Deviation'] = epi_df.std(axis=1)

def calc_eoq(df, cr):
    Z=norm.ppf(cr)
    df['EOQ'] = df['Mean'] + (Z * df['Standard Deviation'])
    return df

def display_eoq(cr):
    # calculate the hospitalization EOQ
    epi_sheet = to_sheet(calc_eoq(epi_df, cr))
    display_population(epi_sheet, round=True)
    # calculate stockpile projections
    preds_df = Data(
        "demand_by_pop_total_pn_tc", config)
    preds_df.array = (demand.demand_by_pop_per_person_pn_uc.array.T * epi_df["EOQ"].to_numpy().T).T
    preds_sheet = to_sheet(preds_df.df)
    display_population(preds_sheet, round=True)

slider = widgets.FloatSlider(min=0.70,max=0.99,step=0.01,value=0.95, continuous_update=False, description="CR")
out = widgets.interactive_output(display_eoq, {'cr': slider})
widgets.VBox([slider, out])

And then this is a rough go at a Vuetify UI - I'll make this prettier and we can package the write-up nicely inside here once that's done.

In [11]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Home']),
    v.Tab(children=['Projected Patients']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[
                    "Hospitalizations due to COVID at a peak surge were projected using several different models. \
                     From this figure, the number of patients requiring ICU care and an invasive ventilator using \
                     IHME figures. The remaining resources were projected using CHA burn rate assumptions." 
                ])
            ]),
            slider, out
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Population"]),
                v.CardSubtitle(children=["Projected Patients"]),
                pop_sheet
            ])
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand"]),
                burn_sheet
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['Home']), Tab(children=['Projected Patients']), Tab(children=['Burn Rates']), Tab…